In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:

import sys
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras import callbacks
import numpy as np

Using TensorFlow backend.


In [0]:

"""
Parameters
"""
epochs = 100
img_width, img_height =300,300
classes_num = 5
batch_size=32
arsitektur = 'Tatsuyah'
#lr = 0.0004

In [0]:

model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode ="same", input_shape=(img_width, img_height, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Convolution2D(64, 2, 2, border_mode ="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering='th'))


model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(classes_num, activation='softmax'))

model.summary()


architecure_dir = os.path.join('/content/gdrive/My Drive/Colab Notebooks/LeafClassification/',arsitektur)
if not os.path.exists(architecure_dir):
  os.mkdir(architecure_dir)
# Save the model architecture
with open(os.path.join(architecure_dir,'architecture.json'), 'w') as f:
    f.write(model.to_json())

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(300, 300,..., padding="same")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (2, 2), padding="same")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
  # Remove the CWD from sys.path while we load stuff.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 300, 300, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 300, 300, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 150, 150, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 150, 150, 64)      8256      
_________________________________________________________________
activation_2 (Activation)    (None, 150, 150, 64)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 150, 75, 32)       0         
___________________________

In [0]:
DATA_DIR = '/content/gdrive/My Drive/CITS Color'

train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    DATA_DIR, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data


Found 1079 images belonging to 5 classes.
Found 268 images belonging to 5 classes.


In [0]:
from keras.models import model_from_json
with open(os.path.join(architecure_dir,'architecture.json'), 'r') as f:
    model = model_from_json(f.read())

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
"""
Tensorboard log
"""


log_dir = os.path.join(architecure_dir, 'tf-log')
tb_cb = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=True, write_images=True)
cbks = [tb_cb]

"""model.fit_generator(
    train_generator,
    samples_per_epoch=samples_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=cbks,
    validation_steps=validation_steps)"""

model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    callbacks=cbks,
    epochs = epochs,
    shuffle = True)

#model.fit_generator(datagen.flow(x_train, y_train, batch_size=32), steps_per_epoch=len(x_train) / 32, epochs=epochs)

target_dir =os.path.join(architecure_dir, 'models')
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
  
model.save(os.path.join(target_dir, 'model.h5'))
model.save_weights(os.path.join(target_dir, 'weights.h5'))

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
33/33 [==============================] - 34s 1s/step - loss: 1.1717 - acc: 0.5170 - val_loss: 0.7406 - val_acc: 0.7266
Epoch 2/100
33/33 [==============================] - 29s 891ms/step - loss: 0.7737 - acc: 0.6927 - val_loss: 0.6140 - val_acc: 0.7881
Epoch 3/100
33/33 [==============================] - 29s 885ms/step - loss: 0.6645 - acc: 0.7549 - val_loss: 0.5065 - val_acc: 0.8178
Epoch 4/100
33/33 [==============================] - 30s 896ms/step - loss: 0.5682 - acc: 0.7642 - val_loss: 0.4179 - val_acc: 0.8644
Epoch 5/100
33/33 [==============================] - 29s 882ms/step - loss: 0.5028 - acc: 0.7988 - val_loss: 0.4043 - val_acc: 0.8390
Epoch 6/100
33/33 [==============================] - 29s 883ms/step - loss: 0.4663 - acc: 0.8337 - val_loss: 0.3606 - val_acc: 0.8729
Epoch 7/100
33/33 [==============================] - 29s 881ms/step - loss: 0.4507 - acc: 0.8271 - val_loss: 0.5071 - val_acc: 0.7924
Epoch 8/100
33/33